<a href="https://colab.research.google.com/github/jjooki/Dacon-Oil/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.decomposition import PCA # 차원축소
from sklearn.model_selection import train_test_split 

from imblearn.under_sampling import * # 임벨런스
from imblearn.over_sampling import * # 임벨런스
from imblearn.combine import * # 임벨런스

import os
import copy
import pandas as pd

from ensemble import BinaryCalssifier, Regressor

In [ ]:
%cd /content/drive/MyDrive/dacon/oil

/content/drive/MyDrive/dacon/oil


In [ ]:
!pwd

/content/drive/MyDrive/dacon/oil


In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 34.6 MB/s 
     |████████████████████████████████| 81 kB 9.7 MB/s 
     |████████████████████████████████| 209 kB 66.2 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
     |████████████████████████████████| 112 kB 81.9 MB/s 
     |████████████████████████████████| 147 kB 69.8 MB/s 
     |████████████████████████████████| 50 kB 5.3 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=3d5bdff69c03a84c9b7f1e625807869f61769f8fe5a9feb1ead8e0d58887975c
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
from jtlearn import Preprocessing
from sklearn.preprocessing import StandardScaler

In [ ]:
save_path = 'submission/'
base_path = 'data/'

train = pd.read_csv(base_path + 'train.csv')
test = pd.read_csv(base_path + 'test.csv')
submission = pd.read_csv(base_path + 'sample_submission.csv')

In [ ]:
train

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,...,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,...,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,...,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,TRAIN_14090,COMPONENT3,1616,2014,8,200,0,2,201,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,135.4,16,0
14091,TRAIN_14091,COMPONENT1,2784,2013,2,200,0,3,85,0,...,NaN,NaN,NaN,NaN,NaN,0,14.5,117.5,1408,0
14092,TRAIN_14092,COMPONENT3,1788,2008,9,550,0,6,0,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,54.0,1301,0
14093,TRAIN_14093,COMPONENT2,2498,2009,19,550,0,2,4,0,...,7.0,8.0,100.0,1625.0,18890.0,0,NaN,44.3,652,0


In [ ]:
test

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,TEST_0000,COMPONENT1,2192,2016,200,0,0,0,1,12,0.0,0,0,0,10,0,0,91.3,1091
1,TEST_0001,COMPONENT3,2794,2011,200,0,0,2,1,278,0.0,3,0,0,2732,1,0,126.9,12
2,TEST_0002,COMPONENT2,1982,2010,200,0,0,0,16,5,0.0,0,0,0,11,0,0,44.3,714
3,TEST_0003,COMPONENT3,1404,2009,200,0,0,3,4,163,0.0,4,3,0,8007,0,0,142.8,94
4,TEST_0004,COMPONENT2,8225,2013,200,0,0,0,6,13,0.0,0,0,0,16,0,0,63.4,469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,TEST_6036,COMPONENT3,1714,2014,200,0,0,3,130,1047,0.0,65,5,2,1999,0,0,71.2,1163
6037,TEST_6037,COMPONENT3,4131,2016,200,0,0,5,2,736,0.0,5,0,0,693,0,0,137.8,13
6038,TEST_6038,COMPONENT3,4325,2014,200,0,0,0,0,53,0.0,0,0,0,69,0,0,302.2,13
6039,TEST_6039,COMPONENT2,1364,2013,200,0,0,0,62,2,0.0,0,0,0,5,0,0,51.1,478


In [ ]:
base_path = '/content/drive/MyDrive/dacon/oil/'
load_path = base_path + 'data/'
save_path = base_path + 'submission/'

train = pd.read_csv(load_path + 'train.csv')
test = pd.read_csv(load_path + 'test.csv')
submission = pd.read_csv(load_path + 'sample_submission.csv')

In [ ]:
# Load Data
save_path = 'submission/'
base_path = 'data/'

train = pd.read_csv(base_path + 'train.csv')
test = pd.read_csv(base_path + 'test.csv')
submission = pd.read_csv(base_path + 'sample_submission.csv')

# Preprocessing

train = train.fillna(0)

X = train.drop(columns=["ID", "Y_LABEL"])
y = train["Y_LABEL"]
test = test.drop(columns=['ID'])

train_num_cols = X.drop(columns=['COMPONENT_ARBITRARY']).columns.tolist()
test_num_cols = test.drop(columns=['COMPONENT_ARBITRARY']).columns.tolist()

ss = StandardScaler()
ss2 = StandardScaler()

ss2.fit(X[test_num_cols])
X[train_num_cols] = ss.fit_transform(X[train_num_cols])
test[test_num_cols] = ss2.transform(test[test_num_cols])

X.COMPONENT_ARBITRARY = X.COMPONENT_ARBITRARY.map({"COMPONENT1" : 1, "COMPONENT2" : 2, "COMPONENT3" : 3, "COMPONENT4" : 4})
test.COMPONENT_ARBITRARY = test.COMPONENT_ARBITRARY.map({"COMPONENT1" : 1, "COMPONENT2" : 2, "COMPONENT3" : 3, "COMPONENT4" : 4})

In [ ]:
X

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,...,U50,U25,U20,U14,U6,U4,V,V100,V40,ZN
0,3,-0.393763,-0.669043,-0.051413,-0.340760,-0.150214,-0.111628,0.281646,-0.238453,-0.041491,...,-0.067643,-0.119657,-0.118804,-0.129231,-0.130716,-0.171264,-0.10655,-0.596701,0.899892,-0.966002
1,2,-0.426022,1.853268,3.715319,-0.022576,-0.150214,-0.123127,-0.437686,-0.238453,-0.041491,...,-0.001086,-0.070899,-0.067218,-0.107725,-0.096483,-0.085911,-0.10655,-0.596701,-1.317376,0.119147
2,2,-0.173409,0.339881,-0.479451,-0.340760,-0.150214,1.118753,-0.612659,0.105735,-0.041491,...,-0.067643,-0.119657,-0.080114,0.010557,1.654031,2.240286,-0.10655,-0.596701,-0.740886,-0.332215
3,3,1.006399,-0.921274,-0.479451,-0.340760,-0.150214,-0.054133,-0.593217,-0.238453,-0.041491,...,-0.067643,-0.119657,-0.118804,-0.129231,-0.130716,-0.171264,-0.10655,-0.596701,0.482642,-1.093888
4,3,0.191634,0.339881,-0.308236,-0.340760,-0.150214,-0.134626,0.903771,-0.238453,-0.041491,...,-0.067643,-0.119657,-0.118804,-0.129231,-0.130716,-0.171264,-0.10655,-0.596701,0.478611,-0.866326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,3,-0.362928,0.087650,0.034194,-0.340760,-0.150214,-0.123127,1.331482,0.105735,-0.041491,...,-0.067643,-0.119657,-0.118804,-0.129231,-0.130716,-0.171264,-0.10655,-0.596701,0.524972,-1.076961
14091,1,-0.085884,-0.164581,-0.479451,-0.340760,-0.150214,-0.111628,0.203880,-0.238453,-0.041491,...,-0.067643,-0.119657,-0.118804,-0.129231,-0.130716,-0.171264,-0.10655,1.815431,0.164162,1.540935
14092,3,-0.322130,-1.425737,0.119802,0.295608,-0.150214,-0.077131,-0.622379,0.105735,-0.041491,...,-0.067643,-0.119657,-0.118804,-0.129231,-0.130716,-0.171264,-0.10655,-0.596701,-1.115806,1.339703
14093,2,-0.153722,-1.173506,0.975877,0.295608,-0.150214,-0.123127,-0.583497,-0.238453,-0.041491,...,-0.067643,0.050996,-0.015631,0.229201,0.126829,0.937623,-0.10655,-0.596701,-1.311328,0.119147


In [ ]:
test

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,1,-0.226304,0.592112,-0.340760,-0.150214,-0.089633,-0.115388,-0.260252,-0.311651,-0.041588,-0.250456,-0.400998,-0.191804,-0.265133,-0.102635,-0.10655,-0.363951,0.944762
1,3,-0.083512,-0.669043,-0.340760,-0.150214,-0.089633,-0.045445,-0.260252,0.187233,-0.041588,0.019051,-0.400998,-0.191804,1.516121,0.042348,-0.10655,0.353638,-1.084484
2,2,-0.276115,-0.921274,-0.340760,-0.150214,-0.089633,-0.115388,-0.143932,-0.324780,-0.041588,-0.250456,-0.400998,-0.191804,-0.264479,-0.102635,-0.10655,-1.311328,0.235748
3,3,-0.413213,-1.173506,-0.340760,-0.150214,-0.089633,-0.010473,-0.236988,-0.028450,-0.041588,0.108886,-0.350857,-0.191804,4.968037,-0.102635,-0.10655,0.674134,-0.930269
4,2,1.204694,-0.164581,-0.340760,-0.150214,-0.089633,-0.115388,-0.221479,-0.309776,-0.041588,-0.250456,-0.400998,-0.191804,-0.261207,-0.102635,-0.10655,-0.926330,-0.225017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,3,-0.339683,0.087650,-0.340760,-0.150214,-0.089633,-0.010473,0.740102,1.629495,-0.041588,5.588859,-0.317430,0.333602,1.036452,-0.102635,-0.10655,-0.769106,1.080171
6037,3,0.233617,0.592112,-0.340760,-0.150214,-0.089633,0.059471,-0.252497,1.046214,-0.041588,0.198722,-0.400998,-0.191804,0.181816,-0.102635,-0.10655,0.573349,-1.082603
6038,3,0.279633,0.087650,-0.340760,-0.150214,-0.089633,-0.115388,-0.268007,-0.234755,-0.041588,-0.250456,-0.400998,-0.191804,-0.226524,-0.102635,-0.10655,3.887155,-1.082603
6039,2,-0.422701,-0.164581,-0.340760,-0.150214,-0.089633,-0.115388,0.212784,-0.330406,-0.041588,-0.250456,-0.400998,-0.191804,-0.268405,-0.102635,-0.10655,-1.174261,-0.208091


In [ ]:
# preparation for preprocessing
sampler_dic = {
    "under": {
        'RandomUnderSampler': RandomUnderSampler,
        'TomekLinks': TomekLinks,
         # 'CondensedNearestNeighbour': CondensedNearestNeighbour, 
        'OneSidedSelection': OneSidedSelection,
        'EditedNearestNeighbours': EditedNearestNeighbours,
        'NeighbourhoodCleaningRule': NeighbourhoodCleaningRule
    },

    "over": {
        'RandomOverSampler': RandomOverSampler,
        'ADASYN': ADASYN,
        'NeighbourhoodCleaningRule': NeighbourhoodCleaningRule
    },

    "hybrid": {
        'SMOTEENN': SMOTEENN,
        'SMOTETomek': SMOTETomek
    }
}
# ("under", "RandomUnderSampler")
# ("over", "RandomOverSampler")
# ("hybrid", "SMOTEENN")
# sampler 하나 
variable_dict = {
    "categorical_feature": "COMPONENT_ARBITRARY", 
    "test_size": 0.2, 
    "learner": ("classification", "LGBM"), 
    "sampler": ("hybrid", "SMOTEENN"), 
    "random_state_": 42,
    "dimensionality": PCA
}

In [ ]:
first_try = Preprocessing(**variable_dict)
print()

# 샘플링 그룹핑 스플릿
X2, y2 = first_try.sampling(X, y)
grouped_dic = first_try.grouping_df(X2, y2, y_column='Y_LABEL')
split_X_y_bundle = first_try.split_X_y_bundle(grouped_dic)
print()


SMOTEENN(random_state=42) completed resampling X and y
COMPONENT_ARBITRARY
dividing my df on 1
dividing my df on 2
dividing my df on 3
dividing my df on 4



In [ ]:
# 피처임포턴스 확인
result_ = first_try.feature_importance_for_groups(split_X_y_bundle)
features = result_[1]
drop_target_list = first_try.chose_drop_features(features, draw=False)
print()
print(drop_target_list)
print()

f1_score : 0.977
f1_score : 0.975
f1_score : 0.983
f1_score : 0.959

['V100', 'BE', 'CD', 'LI', 'U25', 'U4', 'CO', 'V', 'AG', 'FOPTIMETHGLY', 'U14', 'TI', 'FUEL', 'U100', 'U50', 'U20', 'FH2O', 'FSO4', 'FNOX', 'U75', 'FTBN', 'H2O', 'FOXID', 'U6', 'SOOTPERCENTAGE']



In [ ]:
# 파이프라인 결과 확인
print(first_try.print_report(split_X_y_bundle))

SMOTEENN(random_state=42)
LGBMClassifier()
<class 'sklearn.decomposition._pca.PCA'>
PCA() 사용한 pipe line
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       578
           1       0.97      0.99      0.98       633

    accuracy                           0.98      1211
   macro avg       0.98      0.98      0.98      1211
weighted avg       0.98      0.98      0.98      1211

PCA() 사용한 pipe line
              precision    recall  f1-score   support

           0       0.99      0.94      0.96       321
           1       0.96      0.99      0.98       448

    accuracy                           0.97       769
   macro avg       0.97      0.97      0.97       769
weighted avg       0.97      0.97      0.97       769

PCA() 사용한 pipe line
              precision    recall  f1-score   support

           0       0.96      0.93      0.95       972
           1       0.95      0.97      0.96      1309

    accuracy                          

In [ ]:
# # sampler 전부다 
# from tqdm import tqdm

# for key, value in tqdm(sampler_dic.items(), desc="\n첫 번째 반복문"):
#     for name, function in tqdm(value.items(), desc="\n두 번째 반복문"):
#         variable_dict = {
#             "categorical_feature": "COMPONENT_ARBITRARY", 
#             "test_size": 0.1, 
#             "learner": ("classification", "XGB"), 
#             "sampler": (key, name), 
#             "random_state_": 42,
#             "dimensionality": PCA
#         }

#         first_try = Preprocessing(**variable_dict)
#         print()

#         # 샘플링 그룹핑 스플릿
#         X2, y2 = first_try.sampling(X, y)
#         grouped_dic = first_try.grouping_df(X2, y2, y_column='Y_LABEL')
#         split_X_y_bundle = first_try.split_X_y_bundle(grouped_dic)
#         print()

#         # 피처임포턴스 확인
#         result_ = first_try.feature_importance_for_groups(split_X_y_bundle)
#         features = result_[1]
#         drop_target_list = first_try.chose_drop_features(features, draw=False)
#         print()
#         print(drop_target_list)
#         print()

#         # 파이프라인 결과 확인
#         print(first_try.print_report(split_X_y_bundle))
#         break

In [ ]:
drop_target_list

['V100',
 'CD',
 'BE',
 'LI',
 'V',
 'U4',
 'U25',
 'CO',
 'AG',
 'FOPTIMETHGLY',
 'U14',
 'TI',
 'FUEL',
 'SAMPLE_TRANSFER_DAY',
 'U50',
 'U100',
 'U20',
 'FH2O',
 'FSO4',
 'MN',
 'NA',
 'CU',
 'NI',
 'FNOX',
 'CR',
 'U75',
 'FTBN',
 'K',
 'H2O',
 'FOXID',
 'SB',
 'U6',
 'SOOTPERCENTAGE',
 'SN',
 'SI',
 'PB']

In [ ]:
set(drop_target_list) - set(test.columns)

{'BE',
 'CD',
 'FH2O',
 'FNOX',
 'FOPTIMETHGLY',
 'FOXID',
 'FSO4',
 'FTBN',
 'FUEL',
 'K',
 'LI',
 'NA',
 'PB',
 'SAMPLE_TRANSFER_DAY',
 'SB',
 'SI',
 'SN',
 'SOOTPERCENTAGE',
 'U100',
 'U14',
 'U20',
 'U25',
 'U4',
 'U50',
 'U6',
 'U75',
 'V100'}

In [ ]:
set(test.columns) - set(drop_target_list)

{'ANONYMOUS_1',
 'ANONYMOUS_2',
 'COMPONENT_ARBITRARY',
 'FE',
 'MO',
 'PQINDEX',
 'V40',
 'YEAR',
 'ZN'}

In [ ]:
len(drop_target_list)

28

In [ ]:
len(set(drop_target_list) - set(test.columns))

21

In [ ]:
len(set(test.columns) - set(drop_target_list))

11

In [ ]:
print(len(X2.columns), len(test.columns))

52 18


In [ ]:
drop_list_reg = list(set(drop_target_list) - set(test.columns))
drop_list_test = list(set(drop_target_list) & set(test.columns))
X3 = X2.drop(columns=drop_list_reg)
X3

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,CA,...,PB,PQINDEX,S,SB,SI,SN,TI,V,V40,ZN
0,3,-0.393763,-0.669043,-0.051413,-0.340760,-0.150214,-0.111628,0.281646,-0.238453,1.141962,...,-0.160812,5.293270,1.001652,-0.174727,2.006643,0.302478,0.622282,-0.106550,0.899892,-0.966002
1,2,-0.426022,1.853268,3.715319,-0.022576,-0.150214,-0.123127,-0.437686,-0.238453,1.087302,...,0.033010,-0.259244,-1.170187,-0.174727,-0.179489,-0.252439,-0.102635,-0.106550,-1.317376,0.119147
2,3,-0.410367,0.339881,-0.051413,0.415608,-0.150214,-0.146124,-0.418245,-0.238453,-0.834591,...,-0.160812,-0.254663,0.455823,-0.174727,-0.174370,-0.252439,-0.102635,-0.106550,0.789028,-1.061916
3,3,0.411276,0.087650,-0.051413,-0.340760,-0.150214,-0.146124,-0.612659,-0.238453,-0.914895,...,-0.160812,-0.217362,0.421508,-0.174727,-0.169250,-0.252439,-0.102635,-0.106550,0.631804,-0.939672
4,3,-0.468243,1.096575,-0.222628,-0.022576,-0.150214,-0.111628,-0.602938,0.105735,0.646643,...,-0.160812,-0.180717,0.129828,-0.174727,-0.179489,-0.252439,-0.102635,-0.106550,1.299000,-1.099530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22309,1,-0.189589,-1.662718,-0.142197,0.646648,-0.150214,0.624704,-0.565231,0.429113,-0.717098,...,0.693807,-0.262893,-0.825583,0.568990,-0.123017,0.571548,-0.102635,1.996846,0.016880,1.019857
22310,3,-0.209092,0.305027,-0.246288,-0.340760,-0.150214,-0.123127,-0.572115,0.354800,-0.745044,...,-0.160812,-0.213244,-0.733527,-0.174727,-0.144526,-0.013321,-0.102635,-0.106550,-0.941650,1.230105
22311,3,-0.215672,1.801752,-0.495120,2.740946,-0.150214,1.169934,-0.618409,-0.168156,-0.902890,...,-0.083694,-0.180376,0.571305,1.865800,-0.105939,-0.195770,-0.102635,-0.106550,0.371037,-1.072193
22312,1,-0.106748,-1.496417,-0.147370,-0.251598,-0.150214,0.101291,-0.077057,0.504764,-0.254774,...,0.033010,-0.261836,-0.821648,0.088943,-0.134846,0.302478,-0.102635,-0.106550,0.019453,1.274882


In [ ]:
drop_list_test

['AG', 'H2O', 'TI', 'V', 'CO']

In [ ]:
test

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,1,-0.226304,0.592112,-0.340760,-0.150214,-0.089633,-0.115388,-0.260252,-0.311651,-0.041588,-0.250456,-0.400998,-0.191804,-0.265133,-0.102635,-0.10655,-0.363951,0.944762
1,3,-0.083512,-0.669043,-0.340760,-0.150214,-0.089633,-0.045445,-0.260252,0.187233,-0.041588,0.019051,-0.400998,-0.191804,1.516121,0.042348,-0.10655,0.353638,-1.084484
2,2,-0.276115,-0.921274,-0.340760,-0.150214,-0.089633,-0.115388,-0.143932,-0.324780,-0.041588,-0.250456,-0.400998,-0.191804,-0.264479,-0.102635,-0.10655,-1.311328,0.235748
3,3,-0.413213,-1.173506,-0.340760,-0.150214,-0.089633,-0.010473,-0.236988,-0.028450,-0.041588,0.108886,-0.350857,-0.191804,4.968037,-0.102635,-0.10655,0.674134,-0.930269
4,2,1.204694,-0.164581,-0.340760,-0.150214,-0.089633,-0.115388,-0.221479,-0.309776,-0.041588,-0.250456,-0.400998,-0.191804,-0.261207,-0.102635,-0.10655,-0.926330,-0.225017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,3,-0.339683,0.087650,-0.340760,-0.150214,-0.089633,-0.010473,0.740102,1.629495,-0.041588,5.588859,-0.317430,0.333602,1.036452,-0.102635,-0.10655,-0.769106,1.080171
6037,3,0.233617,0.592112,-0.340760,-0.150214,-0.089633,0.059471,-0.252497,1.046214,-0.041588,0.198722,-0.400998,-0.191804,0.181816,-0.102635,-0.10655,0.573349,-1.082603
6038,3,0.279633,0.087650,-0.340760,-0.150214,-0.089633,-0.115388,-0.268007,-0.234755,-0.041588,-0.250456,-0.400998,-0.191804,-0.226524,-0.102635,-0.10655,3.887155,-1.082603
6039,2,-0.422701,-0.164581,-0.340760,-0.150214,-0.089633,-0.115388,0.212784,-0.330406,-0.041588,-0.250456,-0.400998,-0.191804,-0.268405,-0.102635,-0.10655,-1.174261,-0.208091


In [ ]:
resid_cols = X3.drop(columns=test.columns).columns
print('length of residual columns is ', len(resid_cols))
print('residual columns are ', resid_cols)

length of residual columns is  14
residual columns are  Index(['SAMPLE_TRANSFER_DAY', 'AL', 'B', 'BA', 'CA', 'K', 'MG', 'NA', 'P',
       'PB', 'S', 'SB', 'SI', 'SN'],
      dtype='object')


In [ ]:
print('length of test columns is ', len(test.columns))

length of test columns is  18


In [ ]:
test

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,1,-0.226304,0.592112,-0.340760,-0.150214,-0.089633,-0.115388,-0.260252,-0.311651,-0.041588,-0.250456,-0.400998,-0.191804,-0.265133,-0.102635,-0.10655,-0.363951,0.944762
1,3,-0.083512,-0.669043,-0.340760,-0.150214,-0.089633,-0.045445,-0.260252,0.187233,-0.041588,0.019051,-0.400998,-0.191804,1.516121,0.042348,-0.10655,0.353638,-1.084484
2,2,-0.276115,-0.921274,-0.340760,-0.150214,-0.089633,-0.115388,-0.143932,-0.324780,-0.041588,-0.250456,-0.400998,-0.191804,-0.264479,-0.102635,-0.10655,-1.311328,0.235748
3,3,-0.413213,-1.173506,-0.340760,-0.150214,-0.089633,-0.010473,-0.236988,-0.028450,-0.041588,0.108886,-0.350857,-0.191804,4.968037,-0.102635,-0.10655,0.674134,-0.930269
4,2,1.204694,-0.164581,-0.340760,-0.150214,-0.089633,-0.115388,-0.221479,-0.309776,-0.041588,-0.250456,-0.400998,-0.191804,-0.261207,-0.102635,-0.10655,-0.926330,-0.225017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,3,-0.339683,0.087650,-0.340760,-0.150214,-0.089633,-0.010473,0.740102,1.629495,-0.041588,5.588859,-0.317430,0.333602,1.036452,-0.102635,-0.10655,-0.769106,1.080171
6037,3,0.233617,0.592112,-0.340760,-0.150214,-0.089633,0.059471,-0.252497,1.046214,-0.041588,0.198722,-0.400998,-0.191804,0.181816,-0.102635,-0.10655,0.573349,-1.082603
6038,3,0.279633,0.087650,-0.340760,-0.150214,-0.089633,-0.115388,-0.268007,-0.234755,-0.041588,-0.250456,-0.400998,-0.191804,-0.226524,-0.102635,-0.10655,3.887155,-1.082603
6039,2,-0.422701,-0.164581,-0.340760,-0.150214,-0.089633,-0.115388,0.212784,-0.330406,-0.041588,-0.250456,-0.400998,-0.191804,-0.268405,-0.102635,-0.10655,-1.174261,-0.208091


In [ ]:
X_reg = X[test.columns]
X_reg

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,3,-0.393763,-0.669043,-0.340760,-0.150214,-0.089633,0.339245,0.336858,1.331290,-0.041588,1.186914,-0.384284,1.384414,5.293270,0.622282,-0.10655,0.899892,-0.966002
1,2,-0.426022,1.853268,-0.022576,-0.150214,-0.089633,-0.115388,-0.027612,-0.330406,-0.041588,-0.250456,-0.400998,-0.191804,-0.259244,-0.102635,-0.10655,-1.317376,0.119147
2,2,-0.173409,0.339881,-0.340760,-0.150214,-0.089633,-0.080416,-0.252497,-0.326655,-0.041588,-0.250456,-0.400998,-0.191804,-0.260552,-0.102635,-0.10655,-0.740886,-0.332215
3,3,1.006399,-0.921274,-0.340760,-0.150214,-0.089633,-0.115388,-0.260252,-0.264764,-0.041588,-0.160621,-0.400998,-0.191804,-0.242884,-0.102635,-0.10655,0.482642,-1.093888
4,3,0.191634,0.339881,-0.340760,-0.150214,-0.089633,-0.115388,-0.268007,-0.200996,-0.041588,-0.250456,-0.400998,-0.191804,-0.129674,-0.102635,-0.10655,0.478611,-0.866326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,3,-0.362928,0.087650,-0.340760,-0.150214,-0.089633,-0.115388,-0.244743,-0.291021,-0.041588,-0.250456,-0.400998,-0.191804,-0.248773,-0.102635,-0.10655,0.524972,-1.076961
14091,1,-0.085884,-0.164581,-0.340760,-0.150214,-0.089633,-0.115388,-0.252497,-0.311651,-0.041588,-0.250456,3.342831,-0.191804,-0.265787,-0.102635,-0.10655,0.164162,1.540935
14092,3,-0.322130,-1.425737,0.295608,-0.150214,-0.089633,0.024499,-0.213724,0.444177,-0.041588,0.378393,-0.233862,0.070899,0.150406,-0.102635,-0.10655,-1.115806,1.339703
14093,2,-0.153722,-1.173506,0.295608,-0.150214,-0.089633,-0.115388,1.050289,-0.298523,-0.041588,-0.250456,-0.384284,-0.191804,-0.264479,-0.102635,-0.10655,-1.311328,0.119147


In [ ]:
from sklearn.model_selection import train_test_split

reg_dict = {}
for col in resid_cols:
    print(f"--------------- {col} ---------------")
    reg = Regressor()
    y_reg = X[col]
    # train-validation split
    X_reg_train, X_reg_val, y_reg_train, y_reg_val = train_test_split(X_reg, y_reg, test_size=0.2, random_state=69)

    # model fitting
    reg.fit(X_reg_train, y_reg_train, n_trials=20, cv=5)

    # prediction
    y_reg_train_pred = reg.predict(X_reg_train)
    y_reg_pred = reg.predict(X_reg_val)
    
    # scoring
    score_train = reg.score(y_reg_train, y_reg_train_pred, cv=5)
    score_val = reg.score(y_reg_val, y_reg_pred, cv=5)
    reg_dict[col] = reg
    print(f"--------------- {col} ---------------")
    print("Train R^2 score is %.4f" % (score_train))
    print("Validation R^2 score is %.4f" % (score_val))
    print()

[I 2022-11-23 16:32:09,583] A new study created in memory with name: no-name-3e6a5b9e-851e-4634-80a4-d30c85f2a800


--------------- SAMPLE_TRANSFER_DAY ---------------


[I 2022-11-23 16:32:11,698] Trial 0 finished with value: 0.14670738073331677 and parameters: {'n_estimators': 143, 'max_depth': 8, 'min_samples_split': 54, 'min_samples_leaf': 15}. Best is trial 0 with value: 0.14670738073331677.
[I 2022-11-23 16:32:19,849] Trial 1 finished with value: 0.13161943081388938 and parameters: {'n_estimators': 620, 'max_depth': 15, 'min_samples_split': 13, 'min_samples_leaf': 58}. Best is trial 0 with value: 0.14670738073331677.
[I 2022-11-23 16:32:30,332] Trial 2 finished with value: 0.15686522022219035 and parameters: {'n_estimators': 560, 'max_depth': 14, 'min_samples_split': 45, 'min_samples_leaf': 2}. Best is trial 2 with value: 0.15686522022219035.
[I 2022-11-23 16:32:33,490] Trial 3 finished with value: 0.16201919626989442 and parameters: {'n_estimators': 200, 'max_depth': 11, 'min_samples_split': 25, 'min_samples_leaf': 20}. Best is trial 3 with value: 0.16201919626989442.
[I 2022-11-23 16:32:36,813] Trial 4 finished with value: 0.1375238322381591 an

Best trial: score 0.1707559808342673,
params: {'n_estimators': 774, 'max_depth': 16, 'min_samples_split': 33, 'min_samples_leaf': 8}


[I 2022-11-23 16:34:29,186] A new study created in memory with name: no-name-0f4ca2a5-eda4-4417-90dc-1d8e8dd52e9b
[I 2022-11-23 16:34:55,746] Trial 0 finished with value: 0.13192366146044268 and parameters: {'n_estimators': 3620, 'max_depth': 16, 'min_child_weight': 143, 'gamma': 3, 'colsample_bytree': 0.7, 'lambda': 0.04263597731766748, 'alpha': 8.291290565907387, 'subsample': 0.6}. Best is trial 0 with value: 0.13192366146044268.
[I 2022-11-23 16:35:35,445] Trial 1 finished with value: 0.16746727641359227 and parameters: {'n_estimators': 3909, 'max_depth': 8, 'min_child_weight': 71, 'gamma': 1, 'colsample_bytree': 0.6, 'lambda': 0.005046736486729402, 'alpha': 0.21907716723414378, 'subsample': 0.8}. Best is trial 1 with value: 0.16746727641359227.
[I 2022-11-23 16:35:54,886] Trial 2 finished with value: 0.1382588854704882 and parameters: {'n_estimators': 1839, 'max_depth': 16, 'min_child_weight': 271, 'gamma': 2, 'colsample_bytree': 1.0, 'lambda': 0.0029076611925860025, 'alpha': 0.002

Best trial: score 0.21206798984055664,
params: {'n_estimators': 3928, 'max_depth': 14, 'min_child_weight': 11, 'gamma': 1, 'colsample_bytree': 0.9, 'lambda': 0.012363785651462985, 'alpha': 0.0647448260491993, 'subsample': 0.8}


[I 2022-11-23 16:43:27,520] A new study created in memory with name: no-name-58c77a8e-db2b-4dee-a9d7-d245162537e0
[I 2022-11-23 16:43:27,605] Trial 0 finished with value: 0.014609993782144403 and parameters: {'num_leaves': 425, 'max_depth': 1, 'n_estimators': 12, 'class_weight': None, 'min_child_samples': 34, 'subsample': 0.8150305952388075, 'colsample_bytree': 0.7286987693255113, 'reg_alpha': 0.6552766782516062, 'reg_lambda': 5.087631233969013}. Best is trial 0 with value: 0.014609993782144403.
[I 2022-11-23 16:43:27,831] Trial 1 finished with value: -16.2481807731336 and parameters: {'num_leaves': 2, 'max_depth': 7, 'n_estimators': 203, 'class_weight': 'balanced', 'min_child_samples': 28, 'subsample': 0.8961436694171501, 'colsample_bytree': 0.9444188797974001, 'reg_alpha': 0.6097154556054947, 'reg_lambda': 0.060766274161605516}. Best is trial 0 with value: 0.014609993782144403.
[I 2022-11-23 16:43:28,128] Trial 2 finished with value: -6.591714977326868 and parameters: {'num_leaves': 

Best trial: score 0.20723987547471326,
params: {'num_leaves': 256, 'max_depth': 10, 'n_estimators': 369, 'class_weight': None, 'min_child_samples': 34, 'subsample': 0.8309005350959621, 'colsample_bytree': 0.7704730030046895, 'reg_alpha': 0.05001630989001487, 'reg_lambda': 7.986237616028634}


AttributeError: 'Regressor' object has no attribute 'ensemle_'

In [ ]:
import copy
temp_test = copy.deepcopy(test)

for col, reg in reg_dict.items():
    test[col] = reg.predict(temp_test)

X4 = X3.drop(columns=drop_list_test)
test = test.drop(columns=drop_list_test)

In [ ]:
test

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,CR,CU,FE,MN,MO,NI,...,CA,K,MG,NA,P,PB,S,SB,SI,SN
0,1,-0.226304,0.592112,-0.340760,-0.115388,-0.260252,-0.311651,-0.250456,-0.400998,-0.191804,...,0.003382,-0.016572,-0.097588,0.017137,0.015645,-0.000789,-0.269025,-0.025250,-0.059229,-0.010796
1,3,-0.083512,-0.669043,-0.340760,-0.045445,-0.260252,0.187233,0.019051,-0.400998,-0.191804,...,-0.008443,-0.010529,-0.151995,-0.017687,-0.062930,-0.006359,0.277447,-0.022462,0.058414,-0.021918
2,2,-0.276115,-0.921274,-0.340760,-0.115388,-0.143932,-0.324780,-0.250456,-0.400998,-0.191804,...,-0.003565,-0.024774,-0.159044,-0.017687,-0.305593,-0.005957,-0.404327,-0.025049,-0.064950,-0.021918
3,3,-0.413213,-1.173506,-0.340760,-0.010473,-0.236988,-0.028450,0.108886,-0.350857,-0.191804,...,-0.000721,-0.017599,-0.160230,-0.017687,0.601808,-0.006359,0.503987,0.001425,-0.044218,-0.021918
4,2,1.204694,-0.164581,-0.340760,-0.115388,-0.221479,-0.309776,-0.250456,-0.400998,-0.191804,...,-0.008516,-0.024774,-0.161682,-0.017687,-0.381218,-0.006323,-0.246079,-0.025175,-0.063907,-0.021918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,3,-0.339683,0.087650,-0.340760,-0.010473,0.740102,1.629495,5.588859,-0.317430,0.333602,...,0.003741,-0.019292,-0.128025,0.115627,0.117886,0.004786,-0.131196,-0.018005,0.036556,0.275781
6037,3,0.233617,0.592112,-0.340760,0.059471,-0.252497,1.046214,0.198722,-0.400998,-0.191804,...,-0.009219,-0.014332,-0.162290,0.010246,-0.238052,-0.006138,0.267951,-0.022645,-0.013294,-0.021616
6038,3,0.279633,0.087650,-0.340760,-0.115388,-0.268007,-0.234755,-0.250456,-0.400998,-0.191804,...,-0.013841,-0.024774,-0.160090,-0.017687,-0.169606,-0.006359,0.383174,-0.017727,-0.059432,-0.021918
6039,2,-0.422701,-0.164581,-0.340760,-0.115388,0.212784,-0.330406,-0.250456,-0.400998,-0.191804,...,0.008497,-0.021588,-0.161088,-0.017687,-0.342482,-0.005704,-0.306610,-0.025250,-0.064950,-0.014270


In [ ]:
X4

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AL,B,BA,CA,CR,...,NI,P,PB,PQINDEX,S,SB,SI,SN,V40,ZN
0,3,-0.393763,-0.669043,-0.051413,-0.340760,-0.111628,0.281646,-0.238453,1.141962,0.339245,...,1.384414,1.845136,-0.160812,5.293270,1.001652,-0.174727,2.006643,0.302478,0.899892,-0.966002
1,2,-0.426022,1.853268,3.715319,-0.022576,-0.123127,-0.437686,-0.238453,1.087302,-0.115388,...,-0.191804,-0.598302,0.033010,-0.259244,-1.170187,-0.174727,-0.179489,-0.252439,-1.317376,0.119147
2,3,-0.410367,0.339881,-0.051413,0.415608,-0.146124,-0.418245,-0.238453,-0.834591,-0.115388,...,-0.191804,-1.027099,-0.160812,-0.254663,0.455823,-0.174727,-0.174370,-0.252439,0.789028,-1.061916
3,3,0.411276,0.087650,-0.051413,-0.340760,-0.146124,-0.612659,-0.238453,-0.914895,0.024499,...,0.859008,-0.812700,-0.160812,-0.217362,0.421508,-0.174727,-0.169250,-0.252439,0.631804,-0.939672
4,3,-0.468243,1.096575,-0.222628,-0.022576,-0.111628,-0.602938,0.105735,0.646643,-0.115388,...,-0.191804,-1.089115,-0.160812,-0.180717,0.129828,-0.174727,-0.179489,-0.252439,1.299000,-1.099530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22309,1,-0.189589,-1.662718,-0.142197,0.646648,0.624704,-0.565231,0.429113,-0.717098,-0.045445,...,0.062957,-0.015643,0.693807,-0.262893,-0.825583,0.568990,-0.123017,0.571548,0.016880,1.019857
22310,3,-0.209092,0.305027,-0.246288,-0.340760,-0.123127,-0.572115,0.354800,-0.745044,-0.115388,...,-0.191804,0.283263,-0.160812,-0.213244,-0.733527,-0.174727,-0.144526,-0.013321,-0.941650,1.230105
22311,3,-0.215672,1.801752,-0.495120,2.740946,1.169934,-0.618409,-0.168156,-0.902890,-0.045445,...,-0.191804,-1.062114,-0.083694,-0.180376,0.571305,1.865800,-0.105939,-0.195770,0.371037,-1.072193
22312,1,-0.106748,-1.496417,-0.147370,-0.251598,0.101291,-0.077057,0.504764,-0.254774,-0.055245,...,-0.191804,0.254767,0.033010,-0.261836,-0.821648,0.088943,-0.134846,0.302478,0.019453,1.274882


In [ ]:
test

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,CR,CU,FE,MN,MO,NI,...,CA,K,MG,NA,P,PB,S,SB,SI,SN
0,1,-0.226304,0.592112,-0.340760,-0.115388,-0.260252,-0.311651,-0.250456,-0.400998,-0.191804,...,0.003382,-0.016572,-0.097588,0.017137,0.015645,-0.000789,-0.269025,-0.025250,-0.059229,-0.010796
1,3,-0.083512,-0.669043,-0.340760,-0.045445,-0.260252,0.187233,0.019051,-0.400998,-0.191804,...,-0.008443,-0.010529,-0.151995,-0.017687,-0.062930,-0.006359,0.277447,-0.022462,0.058414,-0.021918
2,2,-0.276115,-0.921274,-0.340760,-0.115388,-0.143932,-0.324780,-0.250456,-0.400998,-0.191804,...,-0.003565,-0.024774,-0.159044,-0.017687,-0.305593,-0.005957,-0.404327,-0.025049,-0.064950,-0.021918
3,3,-0.413213,-1.173506,-0.340760,-0.010473,-0.236988,-0.028450,0.108886,-0.350857,-0.191804,...,-0.000721,-0.017599,-0.160230,-0.017687,0.601808,-0.006359,0.503987,0.001425,-0.044218,-0.021918
4,2,1.204694,-0.164581,-0.340760,-0.115388,-0.221479,-0.309776,-0.250456,-0.400998,-0.191804,...,-0.008516,-0.024774,-0.161682,-0.017687,-0.381218,-0.006323,-0.246079,-0.025175,-0.063907,-0.021918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,3,-0.339683,0.087650,-0.340760,-0.010473,0.740102,1.629495,5.588859,-0.317430,0.333602,...,0.003741,-0.019292,-0.128025,0.115627,0.117886,0.004786,-0.131196,-0.018005,0.036556,0.275781
6037,3,0.233617,0.592112,-0.340760,0.059471,-0.252497,1.046214,0.198722,-0.400998,-0.191804,...,-0.009219,-0.014332,-0.162290,0.010246,-0.238052,-0.006138,0.267951,-0.022645,-0.013294,-0.021616
6038,3,0.279633,0.087650,-0.340760,-0.115388,-0.268007,-0.234755,-0.250456,-0.400998,-0.191804,...,-0.013841,-0.024774,-0.160090,-0.017687,-0.169606,-0.006359,0.383174,-0.017727,-0.059432,-0.021918
6039,2,-0.422701,-0.164581,-0.340760,-0.115388,0.212784,-0.330406,-0.250456,-0.400998,-0.191804,...,0.008497,-0.021588,-0.161088,-0.017687,-0.342482,-0.005704,-0.306610,-0.025250,-0.064950,-0.014270


In [ ]:
set(X4.columns) - set(test.columns)

set()

In [ ]:
set(test.columns) - set(X4.columns) 

set()

In [ ]:
test.columns

Index(['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'CR', 'CU',
       'FE', 'MN', 'MO', 'NI', 'PQINDEX', 'V40', 'ZN', 'SAMPLE_TRANSFER_DAY',
       'AL', 'B', 'BA', 'CA', 'K', 'MG', 'NA', 'P', 'PB', 'S', 'SB', 'SI',
       'SN'],
      dtype='object')

In [ ]:
count0 = 0
count1 = 0

for y in y2:
    if y == 0:
        count0 += 1
    elif y == 1:
        count1 += 1

print(count0, count1)

9631 12683


In [ ]:
# sampler 하나 
variable_dict = {
    "categorical_feature": "COMPONENT_ARBITRARY", 
    "test_size": 0.1, 
    "learner": ("classification", "LGBM"), 
    "sampler": ("hybrid", "SMOTEENN"), 
    "random_state_": 42,
    "dimensionality": PCA
}

sec_try = Preprocessing(**variable_dict)
print()

test = test[X4.columns]

# 샘플링 그룹핑 스플릿
grouped_dic = sec_try.grouping_df(X4, y2, y_column='Y_LABEL')        
split_X_y_bundle = first_try.split_X_y_bundle(grouped_dic)

test_final = pd.DataFrame()

for cat, (X_train, X_val, y_train, y_val) in split_X_y_bundle.items():
    test_temp = test[test.COMPONENT_ARBITRARY == cat]
    test_temp = test_temp.drop(columns=['COMPONENT_ARBITRARY'])
    
    # model initializing
    clf = BinaryCalssifier()

    # model training
    clf.fit(X_train, y_train, n_trials=20, cv=5)

    # prediction
    y_train_pred = clf.predict(X_train)
    y_pred = clf.predict(X_val)
    
    # scoring
    score_train = clf.score(y_train, y_train_pred, cv=5)
    score_val = clf.score(y_val, y_pred, cv=5)
    print("Train F1_score is %.4f" % (score_train))
    print("Validation F1_score is %.4f" % (score_val))

    # fill prediction value in test data
    test_temp['Y_LABEL'] = clf.predict(test_temp)
    test_final = pd.concat([test_final, test_temp], axis=0)

test_final = test_final.sort_index()

if 'submission_oil.csv' in os.listdir(save_path):
    count = 0
    for name in os.listdir(save_path):
        if 'submission_oil' in name:
            count += 1
    filename = f"submission_oil{count + 1}.csv"
else:
    filename = 'submission_oil.csv'

# Export submission file
submission.Y_LABEL = test_final.Y_LABEL
submission.to_csv(save_path + filename, index=False)

[I 2022-11-23 15:31:54,205] A new study created in memory with name: no-name-f87b1e46-3434-4c71-941b-faf8feb1664e



COMPONENT_ARBITRARY
dividing my df on 1
dividing my df on 2
dividing my df on 3
dividing my df on 4


[I 2022-11-23 15:31:54,715] Trial 0 finished with value: 0.959829653432352 and parameters: {'num_leaves': 710, 'max_depth': 8, 'n_estimators': 74, 'class_weight': 'balanced', 'min_child_samples': 19, 'subsample': 0.9206740907624216, 'colsample_bytree': 0.8667968973158581, 'reg_alpha': 0.6605415156837734, 'reg_lambda': 7.986051263071242}. Best is trial 0 with value: 0.959829653432352.
[I 2022-11-23 15:31:54,795] Trial 1 finished with value: 0.8749210081961556 and parameters: {'num_leaves': 222, 'max_depth': 3, 'n_estimators': 17, 'class_weight': 'balanced', 'min_child_samples': 12, 'subsample': 0.913247170704724, 'colsample_bytree': 0.7971490563261152, 'reg_alpha': 0.2410905343677674, 'reg_lambda': 8.142874775547053}. Best is trial 0 with value: 0.959829653432352.
[I 2022-11-23 15:31:54,861] Trial 2 finished with value: 0.8113278150712648 and parameters: {'num_leaves': 3, 'max_depth': 4, 'n_estimators': 11, 'class_weight': 'balanced', 'min_child_samples': 23, 'subsample': 0.816909320514

Best trial: score 0.9813214777902441,
params: {'num_leaves': 165, 'max_depth': 7, 'n_estimators': 989, 'class_weight': None, 'min_child_samples': 43, 'subsample': 0.8641125307727103, 'colsample_bytree': 0.7084553134085843, 'reg_alpha': 0.8935153901495289, 'reg_lambda': 2.288787909016112}


[I 2022-11-23 15:32:15,181] A new study created in memory with name: no-name-9866c17a-e082-46db-b542-c37b4a8baa83
[I 2022-11-23 15:32:15,303] Trial 0 finished with value: 0.9268733677832861 and parameters: {'num_leaves': 31, 'max_depth': 4, 'n_estimators': 37, 'class_weight': 'balanced', 'min_child_samples': 42, 'subsample': 0.9245229921723224, 'colsample_bytree': 0.8346053663820154, 'reg_alpha': 0.6544101670477966, 'reg_lambda': 6.056052229639338}. Best is trial 0 with value: 0.9268733677832861.


...

Best trial: score 0.9787026797012892,
params: {'num_leaves': 387, 'max_depth': 10, 'n_estimators': 476, 'class_weight': None, 'min_child_samples': 17, 'subsample': 0.8458077804236227, 'colsample_bytree': 0.7836270672774195, 'reg_alpha': 0.769148151892787, 'reg_lambda': 3.4880057291397186}


[I 2022-11-23 15:33:17,432] A new study created in memory with name: no-name-a22e3b6a-7257-4385-9b2c-fadea2e4b75c


F1_score is 0.9815


[I 2022-11-23 15:33:17,730] Trial 0 finished with value: 0.9291517129834478 and parameters: {'num_leaves': 18, 'max_depth': 3, 'n_estimators': 284, 'class_weight': 'balanced', 'min_child_samples': 19, 'subsample': 0.7225556387788932, 'colsample_bytree': 0.8525034574178451, 'reg_alpha': 0.09172135920083857, 'reg_lambda': 3.552013883854973}. Best is trial 0 with value: 0.9291517129834478.
[I 2022-11-23 15:33:18,478] Trial 1 finished with value: 0.9429959417228952 and parameters: {'num_leaves': 225, 'max_depth': 4, 'n_estimators': 650, 'class_weight': None, 'min_child_samples': 48, 'subsample': 0.8887891049353882, 'colsample_bytree': 0.9033002606109741, 'reg_alpha': 0.104315951319632, 'reg_lambda': 5.292021014777592}. Best is trial 1 with value: 0.9429959417228952.
[I 2022-11-23 15:33:18,720] Trial 2 finished with value: 0.9356263377548257 and parameters: {'num_leaves': 44, 'max_depth': 5, 'n_estimators': 172, 'class_weight': None, 'min_child_samples': 40, 'subsample': 0.8388658173886672,

Best trial: score 0.9517915535011315,
params: {'num_leaves': 930, 'max_depth': 7, 'n_estimators': 999, 'class_weight': None, 'min_child_samples': 21, 'subsample': 0.7773098398538006, 'colsample_bytree': 0.7906525114009102, 'reg_alpha': 0.28288139027850195, 'reg_lambda': 8.37501749710275}
F1_score is 0.9647
